# Лабораторная работа №1 

### Задание 1

Найти набор данных (датасет) для классификации удовлетворяющий следующим условиям: более 10 000 строк, более 20 столбцов, разные типы в столбцах, обязательно наличие целевого признака (таргета).(Документация датасета находится в корне, в файле datasheet.pdf)

### Задание 2

Провести классификацию найденного датасета, методом к- ближайших соседей. В формате Markdown писать пояснения. Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.

### Шаг 1. Подготовка данных

#### Подключение библиотек и вывод таблицы

In [53]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from dateutil.relativedelta import relativedelta

# Загрузка данных
df = pd.read_csv("./Base_v2.csv")

print(df)

       Unnamed: 0 trans_date_trans_time               cc_num  \
0          505859   2019-08-07 17:47:00     4727244663135968   
1          874259   2019-12-21 07:38:24       30143535920989   
2         1004559   2020-02-16 18:24:19     3519232971341141   
3         1083403   2020-03-25 23:20:01     4005676619255478   
4          586261   2019-09-05 23:45:52     4155021259183870   
...           ...                   ...                  ...   
11995      979403   2020-02-02 03:15:02      180048185037117   
11996      368524   2019-06-20 00:08:40       30518206766474   
11997     1049447   2020-03-10 21:50:19         581686439828   
11998        9638   2019-01-06 23:35:09       38544312728630   
11999     1095245   2020-03-31 06:04:13  4769426683924058915   

                                 merchant        category     amt     first  \
0      fraud_O'Connell, Botsford and Hand            home   21.02      Mary   
1                          fraud_Cole PLC     grocery_pos   86.23      Li

### Удаление столбцов с необрабатываемыми типами данных, или которые не влияют на результат мошеничества на карте

In [54]:
columns_to_drop = ['Unnamed: 0', 'merchant','first','last','trans_date_trans_time','trans_num']
df = df.drop(columns=columns_to_drop)
print(df.columns)

Index(['cc_num', 'category', 'amt', 'gender', 'street', 'city', 'state', 'zip',
       'lat', 'long', 'city_pop', 'job', 'dob', 'unix_time', 'merch_lat',
       'merch_long', 'is_fraud'],
      dtype='object')


#### Удаление пустых значений строк

In [55]:
# Найти столбцы с пустыми значениями
empty_columns = df.columns[df.isnull().any()]

# Получить количество пустых значений в каждом столбце
empty_columns_count = df[empty_columns].isnull().sum()

# Вывести информацию о столбцах с пустыми значениями
if len(empty_columns) > 0:
    print("Столбцы, в которых есть пустые значения:")
    for column in empty_columns:
        print(f"Столбец '{column}': {empty_columns_count[column]} пустых значений")
    print()

    # Удалить строки с пустыми значениями
    df_cleaned = df.dropna()
    print(f"Удалено строк с пустыми значениями: {len(df) - len(df_cleaned)}")
else:
    print("Пустых значений нет")

Пустых значений нет


#### Поик нечисловых значений

In [56]:
# Считаем количество столбцов с нечисловыми значениями
non_numeric_columns = df.select_dtypes(exclude=['number']).columns
num_non_numeric_columns = len(non_numeric_columns)

# Вывод информации о нечисловых столбцах
if num_non_numeric_columns > 0:
    print(f"Количество столбцов с нечисловыми значениями: {num_non_numeric_columns}")
    print("Названия столбцов и количество нечисловых значений:")
    for column in non_numeric_columns:
        count_non_numeric = df[column].apply(pd.to_numeric, errors='coerce').isnull().sum()
        print(f"Столбец '{column}': {count_non_numeric} нечисловых значений")
else:
    print("Все столбцы содержат числовые значения")

Количество столбцов с нечисловыми значениями: 7
Названия столбцов и количество нечисловых значений:
Столбец 'category': 12000 нечисловых значений
Столбец 'gender': 12000 нечисловых значений
Столбец 'street': 12000 нечисловых значений
Столбец 'city': 12000 нечисловых значений
Столбец 'state': 12000 нечисловых значений
Столбец 'job': 12000 нечисловых значений
Столбец 'dob': 12000 нечисловых значений


#### Преобразование значимых стлобцов с нечисловыми значениями в числовой формат

In [57]:
# Получаем уникальные значения в столбце 'category'
unique_categories = df['category'].unique()

# Создаем словарь соответствия между уникальными значениями и числовыми представлениями
category_mapping = {category: idx for idx, category in enumerate(unique_categories)}

# Применяем map с использованием словаря
df['category_encoded'] = df['category'].map(category_mapping)

# Выводим результат
print("Уникальные значения в 'category':", unique_categories)
print("\nDataFrame с числовым представлением 'category':")
print(df[['category', 'category_encoded']])


Уникальные значения в 'category': ['home' 'grocery_pos' 'kids_pets' 'gas_transport' 'health_fitness'
 'personal_care' 'shopping_net' 'travel' 'entertainment' 'food_dining'
 'misc_net' 'shopping_pos' 'misc_pos' 'grocery_net']

DataFrame с числовым представлением 'category':
             category  category_encoded
0                home                 0
1         grocery_pos                 1
2                home                 0
3                home                 0
4           kids_pets                 2
...               ...               ...
11995    shopping_net                 6
11996     grocery_pos                 1
11997       kids_pets                 2
11998  health_fitness                 4
11999     grocery_pos                 1

[12000 rows x 2 columns]


In [58]:
# Преобразование 'gender' в числовой формат с использованием replace
df['gender_encoded'] = df['gender'].replace({'M': 1, 'F': 2})

# Выводим результат
print(df[['gender', 'gender_encoded']])

      gender  gender_encoded
0          F               2
1          F               2
2          M               1
3          M               1
4          F               2
...      ...             ...
11995      F               2
11996      F               2
11997      M               1
11998      F               2
11999      F               2

[12000 rows x 2 columns]


In [59]:
# Получаем уникальные значения в столбце 'street'
unique_streets = df['street'].unique()

# Создаем словарь соответствия между уникальными значениями и числовыми представлениями
street_mapping = {street: idx for idx, street in enumerate(unique_streets)}

# Применяем map с использованием словаря
df['street_encoded'] = df['street'].map(street_mapping)

# Выводим результат
print("Уникальные значения в 'street':", unique_streets)
print("\nDataFrame с числовым представлением 'street':")
print(df[['street', 'street_encoded']])

Уникальные значения в 'street': ['118 Justin Extension' '44197 Jeffrey Port Suite 050' '754 Smith Isle'
 '458 Phillips Island Apt. 768' '174 Jennifer Meadow Apt. 467'
 '128 Nathan Ramp' '6340 Jeffrey Dale' '8172 Robertson Parkways Suite 072'
 '8973 Mark Drive' '3603 Mitchell Court' '686 Linda Rest'
 '1561 Chase Grove' '4720 Anna Ways' '582 Rodriguez Wells'
 '1499 Michael Rue' '742 Oneill Shore' '9670 Monroe Junctions'
 '212 Mary Stream' '27823 Andrew Underpass Suite 948'
 '07712 Marcus Divide Apt. 831' '12087 Michael Light'
 '3645 Atkins Island Apt. 238' '821 Solis Points' '517 Vasquez Lodge'
 '2838 White Fields Apt. 473' '6983 Carrillo Isle'
 '246 Stewart Green Suite 149' '269 Sanchez Rapids'
 '838 Franklin Prairie Apt. 902' '4923 Campbell Pines Suite 717'
 '2161 Blankenship Freeway' '524 Wu Spurs Suite 894' '474 Allen Haven'
 '04611 Sandra Spring Suite 059' '133 Alyssa Plains'
 '0207 Griffith Plains Apt. 544' '941 Adam Stravenue'
 '4349 Anna Flat Suite 214' '69252 Oconnor Alley Apt. 

In [60]:
# Получаем уникальные значения в столбце 'city'
unique_cities = df['city'].unique()

# Создаем словарь соответствия между уникальными значениями и числовыми представлениями
city_mapping = {city: idx for idx, city in enumerate(unique_cities)}

# Применяем map с использованием словаря
df['city_encoded'] = df['city'].map(city_mapping)

# Выводим результат
print("Уникальные значения в 'city':", unique_cities)
print("\nDataFrame с числовым представлением 'city':")
print(df[['city', 'city_encoded']])


Уникальные значения в 'city': ['Bay Minette' 'Bridgeport' 'Amsterdam' 'Denham Springs' 'Mountain Park'
 'Barneveld' 'Thornville' 'Superior' 'Stephensport' 'Winfield' 'Kilgore'
 'Conway' 'Murrayville' 'Albuquerque' 'Arlington' 'Florence' 'Rockwood'
 'Arcadia' 'Metairie' 'Laguna Hills' 'Creola' 'Esbon' 'Muskegon'
 'Vancouver' 'Brunson' 'Edisto Island' 'West Monroe' 'Elizabeth'
 'Key West' 'Carlisle' 'West Hartford' 'Mount Hope' 'North Loup' 'Rumely'
 'Lolita' 'Louisiana' 'Nazareth' 'Haynes' 'Ollie' 'Shields'
 'Linthicum Heights' 'Bonfield' 'San Antonio' 'Michigan' 'Lonsdale'
 'Christine' 'Allentown' 'Mount Perry' 'Seneca' 'Houston' 'Bellmore'
 'Port Charlotte' 'Detroit' 'Hedley' 'West Henrietta' 'Easton' 'Harmony'
 'Cuyahoga Falls' 'Pearlington' 'Rocky Mount' 'Lagrange' 'Liberty Mills'
 'Arnold' 'Thompson' 'Big Creek' 'Stittville' 'Tomales' 'Coulee Dam'
 'Barnstable' 'Warren' 'Alton' 'Saint Amant' 'Queen Anne' 'Alexandria'
 'Wilton' 'Carroll' 'Fulton' 'Mifflin' 'Whigham' 'Bauxite' 'Hudso

In [61]:
# Получаем уникальные значения в столбце 'state'
unique_states = df['state'].unique()

# Создаем словарь соответствия между уникальными значениями и числовыми представлениями
state_mapping = {state: idx for idx, state in enumerate(unique_states)}

# Применяем map с использованием словаря
df['state_encoded'] = df['state'].map(state_mapping)

# Выводим результат
print("Уникальные значения в 'state':", unique_states)
print("\nDataFrame с числовым представлением 'state':")
print(df[['state', 'state_encoded']])

Уникальные значения в 'state': ['AL' 'NJ' 'OH' 'LA' 'OK' 'NY' 'AZ' 'KY' 'WV' 'TX' 'NH' 'IL' 'NM' 'VA'
 'MS' 'PA' 'MI' 'CA' 'KS' 'WA' 'SC' 'FL' 'IN' 'VT' 'NE' 'MO' 'AR' 'IA'
 'ND' 'MD' 'MN' 'WI' 'WY' 'UT' 'MA' 'SD' 'GA' 'NC' 'OR' 'RI' 'ME' 'MT'
 'NV' 'TN' 'CT' 'CO' 'ID' 'DC' 'AK' 'HI' 'DE']

DataFrame с числовым представлением 'state':
      state  state_encoded
0        AL              0
1        NJ              1
2        OH              2
3        LA              3
4        OK              4
...     ...            ...
11995    NJ              1
11996    MO             25
11997    PA             15
11998    FL             21
11999    PA             15

[12000 rows x 2 columns]


In [62]:
# Получаем уникальные значения в столбце 'job'
unique_jobs = df['job'].unique()

# Создаем словарь соответствия между уникальными значениями и числовыми представлениями
job_mapping = {job: idx for idx, job in enumerate(unique_jobs)}

# Применяем map с использованием словаря
df['job_encoded'] = df['job'].map(job_mapping)

# Выводим результат
print("Уникальные значения в 'job':", unique_jobs)
print("\nDataFrame с числовым представлением 'job':")
print(df[['job', 'job_encoded']])


Уникальные значения в 'job': ['Science writer' 'Engineer, control and instrumentation'
 'Mental health nurse' 'Herbalist' 'Research scientist (life sciences)'
 'IT trainer' 'Audiological scientist' 'Petroleum engineer'
 'Chartered public finance accountant' 'Exhibition designer'
 'Multimedia programmer' 'Surgeon' 'Commercial/residential surveyor'
 'Civil Service administrator' 'Mudlogger' 'Fine artist'
 'General practice doctor' 'Designer, jewellery'
 'Advertising account planner' 'Sales executive' 'Copywriter, advertising'
 'Tourism officer' 'Historic buildings inspector/conservation officer'
 'Teaching laboratory technician' 'Social researcher'
 'Sales professional, IT' 'Radiographer, diagnostic'
 'Operational researcher' 'Town planner' 'Energy engineer'
 'Development worker, community' 'Librarian, academic' "Nurse, children's"
 'Film/video editor' 'Archaeologist' 'Teacher, early years/pre'
 'Programmer, applications' 'Commercial horticulturist'
 'Conservator, furniture' 'Medical sec

In [63]:
from datetime import datetime

# Преобразование 'dob' в datetime
df['dob'] = pd.to_datetime(df['dob'])

# Расчет возраста и создание нового столбца
df['age'] = (datetime.now() - df['dob']).dt.days // 365

# Выводим результат
print(df[['dob', 'age']])


             dob  age
0     1929-05-30   94
1     1980-08-17   43
2     1961-09-10   62
3     1994-05-31   29
4     1983-10-12   40
...          ...  ...
11995 1974-07-19   49
11996 1979-01-26   44
11997 1973-07-28   50
11998 1991-08-21   32
11999 1954-08-22   69

[12000 rows x 2 columns]


#### Удаление ненужных столбцов, после преобразования данных

In [64]:
columns_to_drop = ['dob','job','street','city','state','gender','category']
df = df.drop(columns=columns_to_drop)
print(df.columns)

Index(['cc_num', 'amt', 'zip', 'lat', 'long', 'city_pop', 'unix_time',
       'merch_lat', 'merch_long', 'is_fraud', 'category_encoded',
       'gender_encoded', 'street_encoded', 'city_encoded', 'state_encoded',
       'job_encoded', 'age'],
      dtype='object')


In [52]:
df.to_csv('base_v3.csv', index=False)

### Шаг 2. Разделение данных и нормализация

In [47]:

# Разделение данных на признаки (X) и целевую переменную (y)
y = df["is_fraud"]
X = df.drop("is_fraud", axis=1)

# Разделение на тренировочные и тестовые данные
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Стандартизация данных
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print(f"Размер обучающего набора: {X_train.shape}")
print(f"Размер тестового набора: {X_test.shape}")



Размер обучающего набора: (9600, 16)
Размер тестового набора: (2400, 16)


###  Подбор гиперпараметров и обучение модели

In [48]:
knn = KNeighborsClassifier()

param_grid = {'n_neighbors': list(range(2, 11)), 'metric': ['euclidean', 'manhattan', 'chebyshev', 'minkowski']}

grid = GridSearchCV(knn, param_grid, refit=True, cv=5,n_jobs=-1)
# Обучение модели
grid.fit(X_train, y_train)

best_params = grid.best_params_
best_knn = grid.best_estimator_

### Обоснование выбора гиперпараметров

In [34]:
# Инициализация и обучение модели методом ближайших соседей
predictions = best_knn.predict(X_test)

# Предсказание на тестовых данных

# Оценка качества модели
accuracy = accuracy_score(y_test, predictions)
report = classification_report(y_test, predictions)

print(f"Accuracy: {accuracy}")
print(f"\nЛучшие параметры: {best_params}")
print("Classification Report:\n", report)

Accuracy: 0.90625

Лучшие параметры: {'metric': 'euclidean', 'n_neighbors': 7}
Classification Report:
               precision    recall  f1-score   support

           0       0.91      0.99      0.95      2038
           1       0.88      0.44      0.59       362

    accuracy                           0.91      2400
   macro avg       0.89      0.71      0.77      2400
weighted avg       0.90      0.91      0.89      2400

